<a href="https://colab.research.google.com/github/Isuru-Pradeep/InteriorWallTextureClassificationAndOtherObjectsColorDetectionModel/blob/main/texture_and__color_detection_model_V2_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Install additional required packages
!pip install yacs opencv-python-headless scikit-image matplotlib pillow
# # Clone the MIT Semantic Segmentation repository
# !git clone https://github.com/CSAILVision/semantic-segmentation-pytorch.git
# %cd semantic-segmentation-pytorch
%cd /content/drive/MyDrive/TextureAndColorDetectionModel/semantic-segmentation-pytorch

# System and file handling
import os
import csv

# Torch and deep learning
import torch
import numpy as np
import torchvision.transforms as transforms
from torchvision import models

# Image handling and processing
from PIL import Image
import scipy.io  # For loading color maps
import matplotlib.pyplot as plt
import cv2  # For additional image processing
from skimage.segmentation import slic
from skimage.color import rgb2lab

# Clustering for color analysis
from sklearn.cluster import KMeans

# Model-specific utilities from MIT Semantic Segmentation repository
from mit_semseg.models import ModelBuilder, SegmentationModule
from mit_semseg.utils import colorEncode

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# Load color map
colors = scipy.io.loadmat('data/color150.mat')['colors']

# Load class names
names = {}
with open('data/object150_info.csv') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        names[int(row[0])] = row[5].split(";")[0]
def visualize_result(img, pred, index=None):
    if index is not None:
        pred = pred.copy()
        pred[pred != index] = -1
        print(f'Showing class: {names[index+1]}')

    pred_color = colorEncode(pred, colors).astype(np.uint8)
    im_vis = np.concatenate((img, pred_color), axis=1)
    display(Image.fromarray(im_vis))
# Create necessary directories
!mkdir -p ckpt/ade20k-resnet50dilated-ppm_deepsup/

# # Download the encoder and decoder weights
# !wget -P ckpt/ade20k-resnet50dilated-ppm_deepsup/ http://sceneparsing.csail.mit.edu/model/pytorch/ade20k-resnet50dilated-ppm_deepsup/encoder_epoch_20.pth
# !wget -P ckpt/ade20k-resnet50dilated-ppm_deepsup/ http://sceneparsing.csail.mit.edu/model/pytorch/ade20k-resnet50dilated-ppm_deepsup/decoder_epoch_20.pth
# Build the encoder and decoder for segmentation
net_encoder = ModelBuilder.build_encoder(
    arch='resnet50dilated',
    fc_dim=2048,
    weights='/content/drive/MyDrive/TextureAndColorDetectionModel/semantic-segmentation-pytorch/ckpt/ade20k-resnet50dilated-ppm_deepsup/encoder_epoch_20.pth'
)
net_decoder = ModelBuilder.build_decoder(
    arch='ppm_deepsup',
    fc_dim=2048,
    num_class=150,
    weights='/content/drive/MyDrive/TextureAndColorDetectionModel/semantic-segmentation-pytorch/ckpt/ade20k-resnet50dilated-ppm_deepsup/decoder_epoch_20.pth',
    use_softmax=True
)

# Create the segmentation module and set to evaluation mode
crit = torch.nn.NLLLoss(ignore_index=-1)
segmentation_module = SegmentationModule(net_encoder, net_decoder, crit)
segmentation_module.eval()
segmentation_module.to(device)


In [ ]:
!pip install ultralytics

In [ ]:
# Required Imports
import numpy as np
import torch
from torchvision import transforms
from PIL import Image
from google.colab import files
import scipy.io
import csv
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy import stats
import cv2
from ultralytics import YOLO  # Import YOLO

# Load YOLO model for texture classification
texture_model = YOLO("/content/drive/MyDrive/TextureAndColorDetectionModel/texture detection model/weights/best.pt")

# Load color map and class names
colors = scipy.io.loadmat('data/color150.mat')['colors']
names = {}
with open('data/object150_info.csv') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        names[int(row[0])] = row[5].split(";")[0]



# Visualize results and perform color analysis on segmented wall
def visualize_result(interiorImageId,email,img, pred, index=None):
    if index is not None:
        pred = pred.copy()
        pred[pred != index] = -1
        print(f'Showing class: {names[index + 1]}')

    # Colorize prediction for visualization
    pred_color = colorEncode(pred, colors).astype(np.uint8)
    im_vis = np.concatenate((img, pred_color), axis=1)
    display(Image.fromarray(im_vis))

    # Isolate wall pixels
    wall_pixels = img[pred == index]
    wall_segmented = np.zeros_like(img)
    wall_segmented[pred == index] = img[pred == index]

    # Step 1: Remove zero-value background to refine the segmented wall region
    mask = np.any(wall_segmented != [0, 0, 0], axis=-1)

    # Check if mask is not empty
    if not np.any(mask):
        print("No valid wall pixels found in the segmented area.")
        return

    # Crop the wall_segmented area to the bounding box of non-zero pixels
    y_indices, x_indices = np.where(mask)
    min_y, max_y = y_indices.min(), y_indices.max()
    min_x, max_x = x_indices.min(), x_indices.max()

    # Extract the non-black region (perfect pixel-occupied portion)
    wall_segmented_cropped = wall_segmented[min_y:max_y+1, min_x:max_x+1]

    # Step 2: Convert cropped segment to PIL Image for YOLO texture classification
    wall_segmented_pil = Image.fromarray(wall_segmented_cropped)

    # Step 3: YOLO Texture Classification
    yolo_results = texture_model.predict(wall_segmented_pil)

    # Step 4: Extract YOLO Classification Results
    texture = "Unknown"
    if len(yolo_results) > 0:
        predicted_class = yolo_results[0].names[yolo_results[0].probs.top1]
        confidence = yolo_results[0].probs.top1conf
        print(f"Texture: {predicted_class} (Confidence: {confidence:.2f})")
        texture = predicted_class

        # Display the refined segmented wall area with texture classification
        plt.figure(figsize=(6, 6))
        plt.imshow(wall_segmented_cropped)
        plt.title(f"Texture Detected: {predicted_class} (Confidence: {confidence:.2f})")
        plt.axis('off')
        plt.show()

    else:
        print("No texture detected by YOLO model.")




     # Reintroduce wall_pixels_refined
    wall_pixels_refined = wall_pixels.reshape(-1, 3)
    # Method 1: K-means Clustering for Dominant Color
    if wall_pixels_refined.size > 0:
      # Check if enough pixels are available for clustering
      if wall_pixels_refined.shape[0] >= 3:  # Ensure at least 3 pixels for 3 clusters
        kmeans = KMeans(n_clusters=3)
        kmeans.fit(wall_pixels_refined)
        avg_dominant_color_kmeans = np.mean(kmeans.cluster_centers_, axis=0).astype(int)

        fig, ax = plt.subplots(1, 2, figsize=(18, 6))

        color_swatch_kmeans = np.zeros((100, 100, 3), dtype=np.uint8)
        color_swatch_kmeans[:, :] = avg_dominant_color_kmeans
        ax[0].imshow(color_swatch_kmeans)
        ax[0].set_title("K-means Dominant Wall Color")
        ax[0].axis('off')

        ax[1].imshow(wall_segmented)
        ax[1].set_title("Segmented Wall Area (Without Edges)")
        ax[1].axis('off')
        plt.show()
      else:
            print(f"Not enough pixels for KMeans clustering for class: {names[index + 1]}")

    # Method 2: Histogram Peak Analysis
    hist, bin_edges = np.histogramdd(wall_pixels.reshape(-1, 3), bins=(8, 8, 8), range=((0, 256), (0, 256), (0, 256)))
    bin_index = np.unravel_index(hist.argmax(), hist.shape)
    avg_dominant_color_hist = np.array([(bin_edges[i][bin_index[i]] + bin_edges[i][bin_index[i] + 1]) / 2 for i in range(3)], dtype=int)

    fig, ax = plt.subplots(1, 2, figsize=(18, 6))


    color_swatch_hist = np.zeros((100, 100, 3), dtype=np.uint8)
    color_swatch_hist[:, :] = avg_dominant_color_hist
    ax[0].imshow(color_swatch_hist)
    ax[0].set_title("Histogram Dominant Wall Color")
    ax[0].axis('off')

    ax[1].imshow(wall_segmented)
    ax[1].set_title("Segmented Wall Area (Without Edges)")
    ax[1].axis('off')
    plt.show()

    # Method 3: Median Color Approach
    median_color = np.median(wall_pixels, axis=0).astype(int)

    fig, ax = plt.subplots(1, 2, figsize=(18, 6))
    color_swatch_median = np.zeros((100, 100, 3), dtype=np.uint8)
    color_swatch_median[:, :] = median_color
    ax[0].imshow(color_swatch_median)
    ax[0].set_title("Median Wall Color")
    ax[0].axis('off')

    ax[1].imshow(wall_segmented)
    ax[1].set_title("Segmented Wall Area (Without Edges)")
    ax[1].axis('off')
    plt.show()



    # Randomly sample  portions from the wall for histogram analysis
    #futher improvement I palned to reduce segemnt area to reduce erros
    num_samples = 2
    sample_size = 5
    samples = []
    sample_coords = []

    # Get coordinates of wall pixels
    wall_coords = np.column_stack(np.where(np.all(wall_segmented != [0, 0, 0], axis=-1)))

    # Randomly select sample positions from wall coordinates
    for _ in range(num_samples):
        coord_index = np.random.choice(len(wall_coords))
        y, x = wall_coords[coord_index]

        # Ensure sample stays within image bounds
        if y + sample_size <= img.shape[0] and x + sample_size <= img.shape[1]:
            sample_portion = wall_segmented[y:y + sample_size, x:x + sample_size]
            samples.append(sample_portion)
            sample_coords.append((x, y))

    img_with_samples = img.copy()
    for (x, y) in sample_coords:
        cv2.rectangle(img_with_samples, (x, y), (x + sample_size, y + sample_size), (255, 0, 0), 2)

    plt.figure(figsize=(6, 6))
    plt.imshow(img_with_samples)
    plt.title("Sampled Portions Highlighted")
    plt.axis('off')
    plt.show()

    # Display histograms for sampled portions
    plt.figure(figsize=(12, 6))
    sampled_colors = []
    for i, sample in enumerate(samples):
      # Compute the histogram
      hist, edges = np.histogramdd(sample.reshape(-1, 3), bins=(8, 8, 8), range=((0, 256), (0, 256), (0, 256)))

      # Perform K-means clustering
      kmeans = KMeans(n_clusters=3)
      kmeans.fit(sample.reshape(-1, 3))
      avg_dominant_color = np.mean(kmeans.cluster_centers_, axis=0).astype(int)

      sampled_colors.append(avg_dominant_color.tolist())

      # Create an image filled with the average dominant color
      color_patch = np.ones_like(sample) * avg_dominant_color

      # Concatenate the original sample with the color patch
      combined_img = np.concatenate((sample, color_patch), axis=1)

      # Display the result
      plt.subplot(2, 5, i + 1)
      plt.imshow(combined_img)
      plt.title(f"Sample {i + 1}\nColor: {avg_dominant_color}")
      plt.axis('off')

    # Collecting data
    data_payload = {
        "interiorImageId":interiorImageId,
        "email":email,
        "texture": texture,
        "wallColorKMeans": avg_dominant_color_kmeans.tolist() if avg_dominant_color_kmeans is not None else None,
        "wallColorHistogram": avg_dominant_color_hist.tolist(),
        "wallColorMedian": median_color.tolist(),
        "sampledColors": sampled_colors
    }

    return data_payload
    # send_data_to_backend(data_payload)

# Visualize results and perform color analysis on other results
def visualize_other_result(interiorImageId,email,img, pred, index=None):
    if index is not None:
        pred = pred.copy()
        pred[pred != index] = -1
        print(f'Showing class: {names[index + 1]}')

    pred_color = colorEncode(pred, colors).astype(np.uint8)
    im_vis = np.concatenate((img, pred_color), axis=1)
    display(Image.fromarray(im_vis))

    # Isolate wall pixels
    wall_pixels = img[pred == index]
    wall_pixels_refined = wall_pixels.reshape(-1, 3)
    wall_segmented = np.zeros_like(img)
    wall_segmented[pred == index] = img[pred == index]

    # Check if any wall pixels are found
    if wall_pixels.size == 0:
        print(f"No pixels found for class: {names[index + 1]}")
        return

    # Method 1: K-means Clustering for Dominant Color
    if wall_pixels_refined.size > 0:
        kmeans = KMeans(n_clusters=3)
        kmeans.fit(wall_pixels_refined)
        avg_dominant_color_kmeans = np.mean(kmeans.cluster_centers_, axis=0).astype(int)

        fig, ax = plt.subplots(1, 2, figsize=(18, 6))
        color_swatch_kmeans = np.zeros((100, 100, 3), dtype=np.uint8)
        color_swatch_kmeans[:, :] = avg_dominant_color_kmeans
        ax[0].imshow(color_swatch_kmeans)
        ax[0].set_title("K-means Dominant Color")
        ax[0].axis('off')

        ax[1].imshow(wall_segmented)
        ax[1].set_title(f"Segmented {names[index + 1]} Area (Without Edges)")
        ax[1].axis('off')
        plt.show()

    # Method 2: Histogram Peak Analysis
    hist, bin_edges = np.histogramdd(wall_pixels.reshape(-1, 3), bins=(8, 8, 8), range=((0, 256), (0, 256), (0, 256)))
    bin_index = np.unravel_index(hist.argmax(), hist.shape)
    avg_dominant_color_hist = np.array([(bin_edges[i][bin_index[i]] + bin_edges[i][bin_index[i] + 1]) / 2 for i in range(3)], dtype=int)

    fig, ax = plt.subplots(1, 2, figsize=(18, 6))
    color_swatch_hist = np.zeros((100, 100, 3), dtype=np.uint8)
    color_swatch_hist[:, :] = avg_dominant_color_hist
    ax[0].imshow(color_swatch_hist)
    ax[0].set_title("Histogram Dominant Color")
    ax[0].axis('off')

    ax[1].imshow(wall_segmented)
    ax[1].set_title(f"Segmented {names[index + 1]} Area (Without Edges)")
    ax[1].axis('off')
    plt.show()

    # Method 3: Median Color Approach
    median_color = np.median(wall_pixels, axis=0).astype(int)

    fig, ax = plt.subplots(1, 2, figsize=(18, 6))
    color_swatch_median = np.zeros((100, 100, 3), dtype=np.uint8)
    color_swatch_median[:, :] = median_color
    ax[0].imshow(color_swatch_median)
    ax[0].set_title("Median Color")
    ax[0].axis('off')

    ax[1].imshow(wall_segmented)
    ax[1].set_title(f"Segmented {names[index + 1]} Area (Without Edges)")
    ax[1].axis('off')
    plt.show()

    # Randomly sample 2 portions from the wall for histogram analysis
    num_samples = 2
    sample_size = 5
    samples = []
    sample_coords = []

    # Get coordinates of wall pixels
    wall_coords = np.column_stack(np.where(np.all(wall_segmented != [0, 0, 0], axis=-1)))

    # Check if wall_coords is not empty
    if len(wall_coords) == 0:
        print("No valid wall coordinates found for sampling.")
        return

    # Randomly select sample positions from wall coordinates
    for _ in range(num_samples):
        coord_index = np.random.choice(len(wall_coords))
        y, x = wall_coords[coord_index]

        # Ensure sample stays within image bounds
        if y + sample_size <= img.shape[0] and x + sample_size <= img.shape[1]:
            sample_portion = wall_segmented[y:y + sample_size, x:x + sample_size]
            samples.append(sample_portion)
            sample_coords.append((x, y))

    img_with_samples = img.copy()
    for (x, y) in sample_coords:
        cv2.rectangle(img_with_samples, (x, y), (x + sample_size, y + sample_size), (255, 0, 0), 2)

    plt.figure(figsize=(6, 6))
    plt.imshow(img_with_samples)
    plt.title(f"Sampled {names[index + 1]} Portions Highlighted")
    plt.axis('off')
    plt.show()

    # Display histograms for sampled portions
    plt.figure(figsize=(12, 6))
    sampled_colors = []
    for i, sample in enumerate(samples):
        # Compute the histogram
        hist, edges = np.histogramdd(sample.reshape(-1, 3), bins=(8, 8, 8), range=((0, 256), (0, 256), (0, 256)))

        # Perform K-means clustering
        kmeans = KMeans(n_clusters=3)
        kmeans.fit(sample.reshape(-1, 3))
        avg_dominant_color = np.mean(kmeans.cluster_centers_, axis=0).astype(int)

        sampled_colors.append(avg_dominant_color.tolist())

        # Create an image filled with the average dominant color
        color_patch = np.ones_like(sample) * avg_dominant_color

        # Concatenate the original sample with the color patch
        combined_img = np.concatenate((sample, color_patch), axis=1)

        # Display the result
        plt.subplot(2, 5, i + 1)
        plt.imshow(combined_img)
        plt.title(f"Sample {i + 1}\nColor: {avg_dominant_color}")
        plt.axis('off')
        plt.show()
    data_payload = {
        "interiorImageId":interiorImageId,
        "email":email,
        "objectName":names[index + 1],
        "ColorKMeans": avg_dominant_color_kmeans.tolist() if avg_dominant_color_kmeans is not None else None,
        "ColorHistogram": avg_dominant_color_hist.tolist(),
        "ColorMedian": median_color.tolist(),
        "sampledColors": sampled_colors
    }
    print(data_payload)
    return data_payload




In [ ]:
pip install requests

In [ ]:
import requests  # Import requests library
from io import BytesIO

# Define the backend URL (update this with your actual EC2 server's URL)
BACKEND_URL = "https://4ddc-2402-d000-a500-4f98-a84a-1566-96e6-2f16.ngrok-free.app/api"

# Helper function to send data to Spring Boot backend
def send_data_to_backend(data):
    try:
        response = requests.post(f"{BACKEND_URL}/saveTextureAndColors", json=data)
        if response.status_code == 200:
            print("Texture and colors data successfully sent to backend.")
        else:
            print(f"Failed to send Texture and colors data. Status Code: {response.status_code}, Response: {response.text}")
    except Exception as e:
        print(f"Error sending data to backend: {e}")

def send_color_data_to_backend(data):
    try:
        response = requests.post(f"{BACKEND_URL}/saveColors", json=data)
        if response.status_code == 200:
            print("Color data successfully sent to backend.")
        else:
            print(f"Failed to send color data. Status Code: {response.status_code}, Response: {response.text}")
    except Exception as e:
        print(f"Error sending data to backend: {e}")

#interior image id and email manually set
interiorImageId =1
email="pradeepisuru31@gmail.com"

# # Upload and open an image
# uploaded = files.upload()
# image_name = list(uploaded.keys())[0]
# pil_image = Image.open(image_name).convert('RGB')
# img_original = np.array(pil_image)

# Upload image using url
# Image URL with parameters
image_url = "https://images.pexels.com/photos/4050284/pexels-photo-4050284.jpeg?auto=compress&cs=tinysrgb&w=1260&h=750&dpr=1"

# Fetch the image from the URL
response = requests.get(image_url, stream=True)
response.raise_for_status()  # Raise an exception for bad responses

# Open the image using PIL
pil_image = Image.open(BytesIO(response.content)).convert('RGB')
img_original = np.array(pil_image)

# Preprocess the image for the segmentation model
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
img_data = transform(pil_image).unsqueeze(0).to(device)

# Run the segmentation model
with torch.no_grad():
    scores = segmentation_module({'img_data': img_data}, segSize=img_data.shape[2:])

# Get the predicted scores for each pixel
_, pred = torch.max(scores, dim=1)
pred = pred.cpu().numpy()[0]

# Define indexes for various objects
wall_index = [k for k, v in names.items() if v == 'wall'][0] - 1

# List of other object indexes for color analysis (excluding walls)
color_detection_indexes = [3, 5, 10, 14, 15, 18, 19, 23, 58, 75] # Adjust indexes according to your segmentation model

# Step 1: Apply texture analysis for the wall
print("\n--- Analyzing Wall Texture and Color ---")
wall_data = visualize_result(interiorImageId,email,img_original, pred, wall_index)
print(wall_data)
send_data_to_backend(wall_data)

# Step 2: Apply color analysis for the specified objects
print("\n--- Analyzing Other Objects' Colors ---")
data_package = []
for obj_index in color_detection_indexes:
    print(f"\nProcessing object index: {obj_index} ({names[obj_index+1]})")
    data = visualize_other_result(interiorImageId,email,img_original, pred, obj_index)
    data_payload = {
        "objectInx":obj_index,
        "color_package": data if data is not None else None
    }
    data_package.append(data_payload)
print(data_package)
#Here is the data end point
send_color_data_to_backend(data_package)

